In [1]:
# Dependencies
import requests
import pandas as pd
import time
import gmaps
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from datetime import datetime
from pprint import pprint
from config import (news_api, g_key)

In [2]:
protest_df = pd.read_csv('Resources/protests.csv')
protest_df

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles
0,2017-01-15,"Bowie State University, Bowie, MD",1500.0,Healthcare,Healthcare; For Affordable Care Act,Yes,http://www.capitalgazette.com/news/ph-ac-cn-ac...,1
1,2017-01-16,"Johnson City, TN",300.0,Civil Rights,Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.johnsoncitypress.com/Local/2017/01/...,4
2,2017-01-16,"Indianapolis, IN",20.0,Environment,Environment; For wilderness preservation,Yes,http://wishtv.com/2017/01/16/nature-groups-pro...,1
3,2017-01-16,"Cincinnati, OH",NaN,Other (Martin Luther King Jr.),Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.cincinnati.com/picture-gallery/news...,1
4,2017-01-18,"Hartford, CT",300.0,Healthcare (Pro-Planned Parenthood),Healthcare; For Planned Parenthood,Yes,http://www.realhartford.org/2017/01/19/reprodu...,1
...,...,...,...,...,...,...,...,...
32518,2020-08-19,"State Capitol, Hartford, CT",20.0,Education,Education; Healthcare; Against hazardous condi...,Yes,https://www.courant.com/coronavirus/hc-news-co...,3
32519,2020-08-19,"Summit Middle School, Summit, NJ",20.0,Education,Education; Healthcare; Against hazardous condi...,Yes,https://www.nj.com/education/2020/08/teachers-...,1
32520,2020-08-19,"East Orange, NJ",100.0,Racial Injustice,Civil Rights; For racial justice; For greater ...,Yes,https://www.pix11.com/news/local-news/nj-stude...,1
32521,2020-08-19,"Gloucester, MA",NaN,Education,Education; Healthcare; Against hazardous condi...,Yes,https://www.gloucestertimes.com/news/local_new...,1


In [4]:
protest_df['Attendees'] = protest_df['Attendees'].fillna(0)
filter_df = protest_df.loc[protest_df['Event (legacy; see tags)'] == 'Racial Injustice'].reset_index(drop=True)
filter_df.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles
0,2017-01-20,"University of Washington, Seattle, WA",0.0,Racial Injustice,Civil Rights; For racial justice; Against invi...,Yes,http://komonews.com/news/local/woman-says-she-...,2
1,2017-01-26,"Ped Mall, Iowa City, IA",30.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://fox59.com/2017/01/30/fedex-backs-driver...,1
2,2017-02-07,"Lebanon Valley College, Annville, PA",50.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://abc27.com/2017/02/07/protestors-push-fo...,1
3,2017-02-09,"University of Michigan, Ann Arbor, MI",150.0,Racial Injustice,Civil Rights; For racial justice; For religiou...,Yes,https://www.michigandaily.com/section/news/stu...,1
4,2017-02-13,"Charlottesville, VA",0.0,Racial Injustice,Civil Rights; For racial justice; Against Conf...,Yes,http://www.wcyb.com/news/politics/va-gubernato...,1


In [ ]:
# google search locations to obtain lat and lng

filter_df['lat'] = ''
filter_df['lng'] = ''

params = {"key": g_key}
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for index, row in filter_df.iterrows():
    params['address'] = row['Location']
    
    lat_lng = requests.get(base_url, params=params).json()
    try:
        filter_df.loc[index, 'lat'] = lat_lng['results'][0]['geometry']['location']['lat']
        filter_df.loc[index, 'lng'] = lat_lng['results'][0]['geometry']['location']['lng']
        print(f"{row['Location']} found")
    except:
        print(f"{row['Location']} could not be found")


In [ ]:
# export to csv for future use
# filter_df.to_csv('filter_df.csv', index=False)

In [10]:
filter_df = pd.read_csv('filter_df.csv')
filter_df.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles,State,lat,lng
0,2017-01-20,"University of Washington, Seattle, WA",0.0,Racial Injustice,Civil Rights; For racial justice; Against invi...,Yes,http://komonews.com/news/local/woman-says-she-...,2,WA,47.655335,-122.303520
1,2017-01-26,"Ped Mall, Iowa City, IA",30.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://fox59.com/2017/01/30/fedex-backs-driver...,1,IA,41.658940,-91.533084
2,2017-02-07,"Lebanon Valley College, Annville, PA",50.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://abc27.com/2017/02/07/protestors-push-fo...,1,PA,40.333741,-76.515080
3,2017-02-09,"University of Michigan, Ann Arbor, MI",150.0,Racial Injustice,Civil Rights; For racial justice; For religiou...,Yes,https://www.michigandaily.com/section/news/stu...,1,MI,42.278044,-83.738224
4,2017-02-13,"Charlottesville, VA",0.0,Racial Injustice,Civil Rights; For racial justice; Against Conf...,Yes,http://www.wcyb.com/news/politics/va-gubernato...,1,VA,38.029306,-78.476678


In [11]:
filter_df['clean_date'] = ''

for index, row in filter_df.iterrows():
    filter_df.loc[index, 'clean_date'] = datetime.strptime(row['Date'], '%Y-%m-%d').strftime('%Y-%m')
filter_df.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles,State,lat,lng,clean_date
0,2017-01-20,"University of Washington, Seattle, WA",0.0,Racial Injustice,Civil Rights; For racial justice; Against invi...,Yes,http://komonews.com/news/local/woman-says-she-...,2,WA,47.655335,-122.303520,2017-01
1,2017-01-26,"Ped Mall, Iowa City, IA",30.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://fox59.com/2017/01/30/fedex-backs-driver...,1,IA,41.658940,-91.533084,2017-01
2,2017-02-07,"Lebanon Valley College, Annville, PA",50.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://abc27.com/2017/02/07/protestors-push-fo...,1,PA,40.333741,-76.515080,2017-02
3,2017-02-09,"University of Michigan, Ann Arbor, MI",150.0,Racial Injustice,Civil Rights; For racial justice; For religiou...,Yes,https://www.michigandaily.com/section/news/stu...,1,MI,42.278044,-83.738224,2017-02
4,2017-02-13,"Charlottesville, VA",0.0,Racial Injustice,Civil Rights; For racial justice; Against Conf...,Yes,http://www.wcyb.com/news/politics/va-gubernato...,1,VA,38.029306,-78.476678,2017-02


In [ ]:
# protest_df['State'] = state.iloc[:,1]

In [17]:
grouped_protest = filter_df.groupby('clean_date')[['Attendees']].sum().reset_index()
grouped_protest.to_csv('grouped_protest.csv', index=False)

In [ ]:
protest_df

In [7]:
filter_

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles,State,lat,lng
0,2017-01-15,"Bowie State University, Bowie, MD",1500.0,Healthcare,Healthcare; For Affordable Care Act,Yes,http://www.capitalgazette.com/news/ph-ac-cn-ac...,1,MD,39.019292,-76.759403
1,2017-01-16,"Johnson City, TN",300.0,Civil Rights,Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.johnsoncitypress.com/Local/2017/01/...,4,TN,36.313440,-82.353473
2,2017-01-16,"Indianapolis, IN",20.0,Environment,Environment; For wilderness preservation,Yes,http://wishtv.com/2017/01/16/nature-groups-pro...,1,IN,39.768403,-86.158068
3,2017-01-16,"Cincinnati, OH",0.0,Other (Martin Luther King Jr.),Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.cincinnati.com/picture-gallery/news...,1,OH,39.103118,-84.512020
4,2017-01-18,"Hartford, CT",300.0,Healthcare (Pro-Planned Parenthood),Healthcare; For Planned Parenthood,Yes,http://www.realhartford.org/2017/01/19/reprodu...,1,CT,41.765804,-72.673372


In [15]:
gmaps.configure(api_key=g_key)
locations = filter_df[["lat", "lng"]]
density = filter_df['Attendees']
density.max()

300000.0

In [16]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=density, 
                                 dissipating=False, max_intensity=50000,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
filter_df['year'] = ''
for index, row in filter_df.iterrows():
    filter_df.loc[index, 'year'] = datetime.strptime(row['Date'], '%Y-%m-%d').strftime('%Y')

In [32]:
filter_df.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles,State,lat,lng,clean_date,year
0,2017-01-20,"University of Washington, Seattle, WA",0.0,Racial Injustice,Civil Rights; For racial justice; Against invi...,Yes,http://komonews.com/news/local/woman-says-she-...,2,WA,47.655335,-122.303520,2017-01,2017
1,2017-01-26,"Ped Mall, Iowa City, IA",30.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://fox59.com/2017/01/30/fedex-backs-driver...,1,IA,41.658940,-91.533084,2017-01,2017
2,2017-02-07,"Lebanon Valley College, Annville, PA",50.0,Racial Injustice,Civil Rights; For racial justice,Yes,http://abc27.com/2017/02/07/protestors-push-fo...,1,PA,40.333741,-76.515080,2017-02,2017
3,2017-02-09,"University of Michigan, Ann Arbor, MI",150.0,Racial Injustice,Civil Rights; For racial justice; For religiou...,Yes,https://www.michigandaily.com/section/news/stu...,1,MI,42.278044,-83.738224,2017-02,2017
4,2017-02-13,"Charlottesville, VA",0.0,Racial Injustice,Civil Rights; For racial justice; Against Conf...,Yes,http://www.wcyb.com/news/politics/va-gubernato...,1,VA,38.029306,-78.476678,2017-02,2017


In [36]:
filter_2017 = filter_df.loc[filter_df['year']=='2017']
filter_2017.head()

gmaps.configure(api_key=g_key)
loc_2017 = filter_2017[["lat", "lng"]]
density = filter_2017['Attendees']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(loc_2017, weights=density, 
                                 dissipating=False, max_intensity=50000,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
filter_2018 = filter_df.loc[filter_df['year']=='2018']
filter_2018.head()

gmaps.configure(api_key=g_key)
loc_2018 = filter_2018[["lat", "lng"]]
density = filter_2018['Attendees']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(loc_2018, weights=density, 
                                 dissipating=False, max_intensity=50000,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
filter_2019 = filter_df.loc[filter_df['year']=='2019']
filter_2019.head()

gmaps.configure(api_key=g_key)
loc_2019 = filter_2019[["lat", "lng"]]
density = filter_2019['Attendees']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(loc_2019, weights=density, 
                                 dissipating=False, max_intensity=50000,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
filter_2020 = filter_df.loc[filter_df['year']=='2020']
filter_2020.head()

gmaps.configure(api_key=g_key)
loc_2020 = filter_2020[["lat", "lng"]]
density = filter_2020['Attendees']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(loc_2020, weights=density, 
                                 dissipating=False, max_intensity=50000,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
filter_df['Attendees'].max()

In [ ]:
protest_df['clean_date'] =''
for index, row in protest_df.iterrows():
    protest_df.loc[index, 'clean_date'] = datetime.strptime(row['Date'], '%Y-%m-%d').date()

In [ ]:
filter_df = protest_df.loc[protest_df['Event (legacy; see tags)'] =='Racial Injustice']
filter_df

In [ ]:
filter_df.plot.line(x='clean_date', y='Attendees')

In [ ]:
protest_df.plot.scatter(x='lng', y='lat')